In [1]:
using Pkg
Pkg.activate(".")
using SymPy

  Activating project at `c:\Users\tim\Documents\2025FallExamples\10-10`


# Afterclass wrap up
The last example problem from class was based on Greenwood P6-9, with some modifications to be a 2-dof problem.

The work here is just on the part of the problem that we got through during class

In [2]:
g, l, m, J, r, t = symbols("g l m J r t", real=true, positive=true)
ϕ = SymFunction("phi", real=true)(t)
θ = SymFunction("theta", real=true)(t)
p1 = SymFunction("p_1", real=true)(t)
p2 = SymFunction("p_2", real=true)(t)

ihat = [1, 0, 0]
jhat = [0, 1, 0]
khat = [0, 0, 1]
Dot(a, b) = sum(a .* b);

In [3]:
R = r*(sin(ϕ)*ihat - cos(ϕ)*jhat) + l*(sin(θ)*ihat - cos(θ)*jhat)

3-element Vector{Sym{PyCall.PyObject}}:
  l*sin(theta(t)) + r*sin(phi(t))
 -l*cos(theta(t)) - r*cos(phi(t))
                                0

In [4]:
V = -m*g*Dot(R,jhat)

-g*m*(-l*cos(theta(t)) - r*cos(phi(t)))

In [5]:
T = 1//2*J*diff(ϕ,t)^2 + 1//2*m*Dot(diff.(R,t),diff.(R,t)) |> simplify

              2     /                 2                                        >
  /d         \      | 2 /d           \                                 d       >
J*|--(phi(t))|    m*|l *|--(theta(t))|  + 2*l*r*cos(phi(t) - theta(t))*--(phi( >
  \dt        /      \   \dt          /                                 dt      >
--------------- + ------------------------------------------------------------ >
       2                                                         2             >

>                                   2\
>     d               2 /d         \ |
> t))*--(theta(t)) + r *|--(phi(t))| |
>     dt                \dt        / /
> ------------------------------------
>                                     

In [6]:
L = T-V

              2                                              /                 >
  /d         \                                               | 2 /d            >
J*|--(phi(t))|                                             m*|l *|--(theta(t)) >
  \dt        /                                               \   \dt           >
--------------- + g*m*(-l*cos(theta(t)) - r*cos(phi(t))) + ------------------- >
       2                                                                       >

>  2                                                                         2 >
> \                                 d          d               2 /d         \  >
> |  + 2*l*r*cos(phi(t) - theta(t))*--(phi(t))*--(theta(t)) + r *|--(phi(t))|  >
> /                                 dt         dt                \dt        /  >
> ---------------------------------------------------------------------------- >
>                             2                                                >

> \
> |
> |
> /
> -
>  

In [7]:
eq1 = -p1 + diff(L, diff(ϕ,t)) |> simplify

  d                /                         d                d         \      >
J*--(phi(t)) + m*r*|l*cos(phi(t) - theta(t))*--(theta(t)) + r*--(phi(t))| - p_ >
  dt               \                         dt               dt        /      >

>     
> 1(t)
>     

In [8]:
eq2 = -p2 + diff(L, diff(θ, t)) |> simplify

    /  d                                       d         \         
l*m*|l*--(theta(t)) + r*cos(phi(t) - theta(t))*--(phi(t))| - p_2(t)
    \  dt                                      dt        /         

In [9]:
sol_p = solve([eq1, eq2], [diff(ϕ,t), diff(θ,t)])

Dict{Sym{PyCall.PyObject}, Sym{PyCall.PyObject}} with 2 entries:
  Derivative(phi(t), t)   => -l*p_1(t)/(-J*l + l*m*r^2*cos(phi(t) - theta(t))^2…
  Derivative(theta(t), t) => -J*p_2(t)/(-J*l^2*m + l^2*m^2*r^2*cos(phi(t) - the…

## continuing on

In [14]:
H = subs(T + V, sol_p) |> simplify

                                                           2                   >
       3  2                      2  2                 J*p_2 (t)      3  3  2   >
- J*g*l *m *cos(theta(t)) - J*g*l *m *r*cos(phi(t)) - --------- + g*l *m *r *c >
                                                          2                    >
------------------------------------------------------------------------------ >
                                                                               >
                                                                               >

>                                                                              >
>   2                                       3  3  2                    2  3  3 >
> os (phi(t) - theta(t))*cos(theta(t)) - g*l *m *r *cos(theta(t)) + g*l *m *r  >
>                                                                              >
> ---------------------------------------------------------------------------- >
>                          

In [16]:
pdot = [-diff(H,q) |> simplify for q in [ϕ,θ]]

2-element Vector{Sym{PyCall.PyObject}}:
                 r*(J^2*g*l^2*m*sin(phi(t)) - 2*J*g*l^2*m^2*r^2*sin(phi(t))*cos(phi(t) - theta(t))^2 + 2*J*g*l^2*m^2*r^2*sin(phi(t)) - J*l*p_1(t)*p_2(t)*sin(phi(t) - theta(t)) + J*r*p_2(t)^2*sin(2*phi(t) - 2*theta(t))/2 + g*l^2*m^3*r^4*sin(phi(t))*cos(phi(t) - theta(t))^4 - 2*g*l^2*m^3*r^4*sin(phi(t))*cos(phi(t) - theta(t))^2 + g*l^2*m^3*r^4*sin(phi(t)) + l^2*m*r*p_1(t)^2*sin(2*phi(t) - 2*theta(t))/2 - l*m*r^2*p_1(t)*p_2(t)*sin(phi(t) - theta(t))*cos(phi(t) - theta(t))^2 - l*m*r^2*p_1(t)*p_2(t)*sin(phi(t) - theta(t)) + m*r^3*p_2(t)^2*sin(2*phi(t) - 2*theta(t))/2)/(l^2*(J^2 - 2*J*m*r^2*cos(phi(t) - theta(t))^2 + 2*J*m*r^2 + m^2*r^4*cos(phi(t) - theta(t))^4 - 2*m^2*r^4*cos(phi(t) - theta(t))^2 + m^2*r^4))
 (J^2*g*l^3*m*sin(theta(t)) - 2*J*g*l^3*m^2*r^2*sin(theta(t))*cos(phi(t) - theta(t))^2 + 2*J*g*l^3*m^2*r^2*sin(theta(t)) + J*l*r*p_1(t)*p_2(t)*sin(phi(t) - theta(t)) - J*r^2*p_2(t)^2*sin(2*phi(t) - 2*theta(t))/2 + g*l^3*m^3*r^4*sin(theta(t))*cos(p

In [17]:
qdot = [ +diff(H,p) |> simplify for p in [p1,p2] ]

2-element Vector{Sym{PyCall.PyObject}}:
                        (l*p_1(t) - r*p_2(t)*cos(phi(t) - theta(t)))/(l*(J - m*r^2*cos(phi(t) - theta(t))^2 + m*r^2))
 (J*p_2(t) - l*m*r*p_1(t)*cos(phi(t) - theta(t)) + m*r^2*p_2(t))/(l^2*m*(J - m*r^2*cos(phi(t) - theta(t))^2 + m*r^2))